<a href="https://colab.research.google.com/github/anonyblank/ml-zoomcamp-hw/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv'

--2023-09-29 12:59:06--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: ‘data.csv.14’

data.csv.14         100%[===================>]   1.41M  6.15MB/s    in 0.2s    

2023-09-29 12:59:06 (6.15 MB/s) - ‘data.csv.14’ saved [1475504/1475504]



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
raw_datasets = pd.read_csv('data.csv')

In [4]:
selected_cols = [
    'Make',
    'Model',
    'Year',
    'Engine HP',
    'Engine Cylinders',
    'Transmission Type',
    'Vehicle Style',
    'highway MPG',
    'city mpg',
    'MSRP'
]

In [5]:
datasets = raw_datasets[selected_cols]

In [6]:
datasets.columns = datasets.columns.str.replace(' ', '_').str.lower()

In [7]:
datasets = datasets.fillna(0)

In [8]:
datasets = datasets.rename(columns={"msrp": "price"})

In [9]:
numerical = [
    'year',
    'engine_hp',
    'engine_cylinders',
    'highway_mpg',
    'city_mpg'
]

In [10]:
data_num = datasets.copy()
data_num = data_num[numerical]
data_num.describe()

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg
count,11914.000000,11914.000000,11914.000000,11914.000000,11914.000000
mean,2010.384338,247.941749,5.614655,26.637485,19.733255
std,7.579740,110.507669,1.800554,8.863001,8.987798
min,1990.000000,0.000000,0.000000,12.000000,7.000000
25%,2007.000000,170.000000,4.000000,22.000000,16.000000
50%,2015.000000,225.000000,6.000000,26.000000,18.000000
75%,2016.000000,300.000000,6.000000,30.000000,22.000000
max,2017.000000,1001.000000,16.000000,354.000000,137.000000


In [11]:
datasets.transmission_type.mode()

0    AUTOMATIC
Name: transmission_type, dtype: object

In [12]:
# Bulid in correlation_matrix
correlation_matrix = data_num.corr()
correlation_matrix

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg
year,1.000000,0.338714,-0.040708,0.258240,0.198171
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000


In [13]:
# Own correlation_matrix
corr_matrix = pd.DataFrame(index=numerical)

for numeric in numerical:
  corr_matrix[numeric] = datasets[numerical].corrwith(datasets[numeric]).values

corr_matrix

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg
year,1.000000,0.338714,-0.040708,0.258240,0.198171
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000


In [14]:
biggest_correlation = data_num.corr().unstack().sort_values(ascending=False)[:15]
biggest_correlation

year              year                1.000000
engine_hp         engine_hp           1.000000
highway_mpg       highway_mpg         1.000000
engine_cylinders  engine_cylinders    1.000000
city_mpg          city_mpg            1.000000
                  highway_mpg         0.886829
highway_mpg       city_mpg            0.886829
engine_hp         engine_cylinders    0.774851
engine_cylinders  engine_hp           0.774851
engine_hp         year                0.338714
year              engine_hp           0.338714
                  highway_mpg         0.258240
highway_mpg       year                0.258240
city_mpg          year                0.198171
year              city_mpg            0.198171
dtype: float64

In [15]:
data_class = datasets.copy()
mean = data_class['price'].mean()
data_class['average_above'] = np.where(data_class['price'] >= mean, 1, 0)

In [16]:
data_class = data_class.drop('price', axis=1)

In [17]:
# datasets["average_above"] = (datasets.price >= datasets.price.mean()).astype(int).values

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
datasets_for_full_train, datasets_for_test = train_test_split(data_class, test_size=0.2, random_state=42)

In [20]:
datasets_for_train, datasets_for_val = train_test_split(datasets_for_full_train, test_size=0.25, random_state=42)

In [21]:
len(datasets_for_train + datasets_for_val + datasets_for_test), len(datasets)

(11914, 11914)

In [22]:
datasets_for_full_train = datasets_for_full_train.reset_index(drop=True)
datasets_for_train = datasets_for_train.reset_index(drop=True)
datasets_for_val = datasets_for_val.reset_index(drop=True)
datasets_for_test = datasets_for_test.reset_index(drop=True)

In [23]:
labels_for_train = datasets_for_train.average_above.values
labels_for_val = datasets_for_val.average_above.values
labels_for_test = datasets_for_test.average_above.values

In [24]:
categorical = [
    'make',
    'model',
    'transmission_type',
    'vehicle_style'
]

In [25]:
from sklearn.metrics import mutual_info_score

In [26]:
def mutual_info_average_score(series):
  return mutual_info_score(datasets_for_train.average_above, series)

score = datasets_for_train[categorical].apply(mutual_info_average_score).sort_values(ascending=True)
round(score, 2)

transmission_type    0.02
vehicle_style        0.08
make                 0.24
model                0.46
dtype: float64

In [27]:
datasets_for_train = datasets_for_train.drop('average_above', axis=1)
datasets_for_val = datasets_for_val.drop('average_above', axis=1)
datasets_for_test = datasets_for_test.drop('average_above', axis=1)

In [28]:
from sklearn.feature_extraction import DictVectorizer

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
from sklearn.metrics import accuracy_score

In [31]:
dv = DictVectorizer(sparse=False)

dict_datasets_for_train = datasets_for_train[categorical+numerical].to_dict(orient='records')
dv.fit(dict_datasets_for_train)
features_for_train = dv.transform(dict_datasets_for_train)

dict_datasets_for_val = datasets_for_val[categorical+numerical].to_dict(orient='records')
features_for_val = dv.transform(dict_datasets_for_val)

In [32]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(features_for_train,labels_for_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [33]:
pred_labels_for_val = model.predict(features_for_val)
pred_labels_for_val

array([0, 1, 0, ..., 0, 1, 1])

In [34]:
accuracy = np.round(accuracy_score(labels_for_val, pred_labels_for_val),2)
accuracy

0.93

In [35]:
org_score = accuracy
features_list = categorical + numerical

for feature in features_list:

  features = features_list.copy()
  features.remove(feature)

  dict_datasets_for_train = datasets_for_train[features].to_dict(orient='records')
  dv = DictVectorizer(sparse=False)

  dv.fit(dict_datasets_for_train)
  features_for_train = dv.transform(dict_datasets_for_train)

  model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
  model.fit(features_for_train, labels_for_train)

  dict_datasets_for_val = datasets_for_val[features].to_dict(orient='records')
  features_for_val = dv.transform(dict_datasets_for_val)

  pred_labels_for_val = model.predict(features_for_val)

  score = accuracy_score(labels_for_val, pred_labels_for_val)
  print(feature, org_score - score, org_score, score)

make -0.018804028535459483 0.93 0.9488040285354595
model 0.013508182962652215 0.93 0.9164918170373478
transmission_type -0.015027276542173729 0.93 0.9450272765421738
vehicle_style -0.002018464120856045 0.93 0.9320184641208561
year -0.01838438942509435 0.93 0.9483843894250944
engine_hp -0.004955937893411644 0.93 0.9349559378934117
engine_cylinders -0.01712547209399906 0.93 0.9471254720939991
highway_mpg -0.01670583298363404 0.93 0.9467058329836341
city_mpg -0.015866554762903884 0.93 0.9458665547629039


In [36]:
# year -0.013848090642047839 0.9483843894250944 0.9345362987830466
# engine_hp -0.00041963911036513313 0.9349559378934117 0.9345362987830466
# transmission_type -0.010490977759127218 0.9450272765421738 0.9345362987830466
# city_mpg -0.011330255979857373 0.9458665547629039 0.9345362987830466

In [37]:
datasets['price'] = np.log1p(datasets['price'])
datasets['price'] = np.log1p(datasets['price'])
datasets['price'] = np.log1p(datasets['price'])

In [38]:
datasets

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,1.242120
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,1.238985
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,1.236178
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,1.230798
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,1.234855
...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,1.242112
11910,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,1.247123
11911,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,1.244390
11912,Acura,ZDX,2013,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,1.244534


In [39]:
datasets_for_full_train, datasets_for_test = train_test_split(datasets, test_size=0.2, random_state=42)
datasets_for_train, datasets_for_val = train_test_split(datasets_for_full_train, test_size=0.25, random_state=42)

In [40]:
datasets_for_train = datasets_for_train.reset_index(drop=True)
datasets_for_val = datasets_for_val.reset_index(drop=True)
datasets_for_test = datasets_for_test.reset_index(drop=True)

In [41]:
labels_for_train =  datasets_for_train.price.values
labels_for_val =  datasets_for_val.price.values
labels_for_test =  datasets_for_test.price.values

In [42]:
del datasets_for_train['price']
del datasets_for_val['price']
del datasets_for_test['price']

In [43]:
dv = DictVectorizer(sparse=False)

dict_datasets_for_train = datasets_for_train[categorical+numerical].to_dict(orient='records')
dv.fit(dict_datasets_for_train)
features_for_train = dv.transform(dict_datasets_for_train)

dict_datasets_for_val = datasets_for_val[categorical+numerical].to_dict(orient='records')
features_for_val = dv.transform(dict_datasets_for_val)


In [44]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [45]:
import warnings
warnings.filterwarnings('ignore')
alpha_list = [0, 0.01, 0.1, 1, 10]

for alpha in alpha_list:
  model_r = Ridge(alpha=alpha, solver='sag', random_state=42)
  model_r.fit(features_for_train, labels_for_train)

  pred_labels_for_val = model_r.predict(features_for_val)

  rmse = np.sqrt(mean_squared_error(labels_for_val, pred_labels_for_val))

  print(alpha, round(rmse, 3))
# # 0.4867943132423885
# # 0.4867945519275268
# # 0.48679670001899794
# # 0.4868181745432731
# # 0.48703228329751225

0 0.014
0.01 0.014
0.1 0.014
1 0.014
10 0.014
